### Bibliotecas

In [695]:
import geopandas as gpd
import pandas as pd
import cx_Oracle
import os
from dotenv import load_dotenv
from shapely.wkt import dumps
from shapely.geometry import Point, Polygon
from shapely.wkt import loads
from shapely import wkt

### Diretórios

In [696]:
# DEFINIÇÕES DE DIRETÓRIOS
BASE_DIR = os.getcwd()
DIR_PAI = os.path.dirname(os.path.dirname(BASE_DIR))
DIR_COLETA_DADOS = os.path.join(DIR_PAI, "Coleta de Dados")
DIR_CORD = os.path.join(DIR_COLETA_DADOS, "COORDENADAS_REGIOES")
DIR_MUNICIPIO_SP = os.path.join(DIR_CORD, "cidade_sp.geojson")
DIR_ZONAS_SP = os.path.join(DIR_CORD, "zonas_sp.geojson")
DIR_DISTRITOS_SP = os.path.join(DIR_CORD, "distrito_sp.geojson")

In [697]:
load_dotenv()

True

In [698]:
username = os.getenv('USERNAME_ORC')
password = os.getenv("PASSWORD_ORC")
dsn = 'ORACLE.FIAP.COM.BR:1521/ORCL'

### Funções Auxiliares

In [699]:
def conexao_oracle(username, password, dsn):
    connection = None
    cursor = None
    try:
        # Adiciona o caminho do Instant Client manualmente, caso o PATH não funcione
        os.environ['PATH'] = os.getenv('PATH_instantclient_23_5') + ";" + os.environ['PATH']
        
        connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM TBL_Previsao")

        for row in cursor:
            print(row)

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao conectar-se ao Oracle: {error.message}")

    finally:
        return connection, cursor

In [700]:
def finaliza_conexao(cursor, connection):
    if cursor:
        cursor.close()
    if connection:
        connection.close()

In [701]:
def consulta_para_dataframe(cursor, query):
    try:
        # Executa a consulta
        cursor.execute(query)
        
        # Pega os nomes das colunas
        colunas = [col[0] for col in cursor.description]
        
        # Obtém os dados da consulta
        dados = cursor.fetchall()
        
        # Cria o DataFrame
        df = pd.DataFrame(dados, columns=colunas)
        
        return df
    
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao executar a consulta: {error.message}")
        return None

#### Inicializando Conexão ORC

In [702]:
connection, cursor = conexao_oracle(username, password, dsn)

#if connection and cursor:
#    finaliza_conexao(cursor, connection)

#### INSERT COORDENADAS - TBL_Municipio, TBL_Zona, TBL_Distrito 

##### TBL_Municipio

In [703]:
# Consulta SQL
query_teste = "SELECT * FROM TBL_Municipio"

# Executa a consulta e obtém os resultados em um DataFrame
df_teste = consulta_para_dataframe(cursor, query_teste)
df_teste

,CD_MUN,NM_MUN,SG_ESTADO,AREA_KM2,GEOMETRY,CORD_CENTRAL


In [704]:
# Carregar o arquivo GeoJSON usando GeoPandas
gdf_mun = gpd.read_file(DIR_MUNICIPIO_SP)

# Verifique o conteúdo do GeoDataFrame
print(gdf_mun.head())
print(gdf_mun.dtypes)

    CD_MUN     NM_MUN  AREA_KM2                   CORD_CENTRAL  \
0  3550308  São Paulo   1521.11  POINT (-46.647972 -23.649892)   

                                            geometry  
0  POLYGON ((-46.55066 -23.35714, -46.55011 -23.3...  
CD_MUN             int32
NM_MUN            object
AREA_KM2         float64
CORD_CENTRAL      object
geometry        geometry
dtype: object


In [705]:
def insert_municipio_data(connection, gdf):
    cursor = connection.cursor()

    try:
        for _, row in gdf.iterrows():
            cd_mun = row['CD_MUN']
            nm_mun = row['NM_MUN']
            sg_estado = 'SP'  # Ajuste conforme necessário
            area_km2 = row['AREA_KM2']

            # Certifique-se de que as colunas 'geometry' e 'CORD_CENTRAL' são objetos Shapely
            if isinstance(row['geometry'], str):
                geom = loads(row['geometry'])
            else:
                geom = row['geometry']

            if isinstance(row['CORD_CENTRAL'], str):
                central_geom = loads(row['CORD_CENTRAL'])
            else:
                central_geom = row['CORD_CENTRAL']

            # Converta para WKT para ser utilizado no PL/SQL
            geometry_wkt = geom.wkt
            cor_central_wkt = central_geom.wkt if central_geom else None

            # Bloco PL/SQL para inserir a geometria no Oracle
            plsql = """
            DECLARE
                geom SDO_GEOMETRY;
                central_geom SDO_GEOMETRY;
            BEGIN
                geom := SDO_GEOMETRY(:geometry_wkt, 4326);
                IF :cor_central_wkt IS NOT NULL THEN
                    central_geom := SDO_GEOMETRY(:cor_central_wkt, 4326);
                ELSE
                    central_geom := NULL;
                END IF;
                INSERT INTO TBL_Municipio (CD_MUN, NM_MUN, SG_Estado, AREA_KM2, GEOMETRY, CORD_CENTRAL)
                VALUES (:cd_mun, :nm_mun, :sg_estado, :area_km2, geom, central_geom);
            END;
            """
            cursor.execute(plsql, cd_mun=cd_mun, nm_mun=nm_mun, sg_estado=sg_estado, area_km2=area_km2, 
                           geometry_wkt=geometry_wkt, cor_central_wkt=cor_central_wkt)

        connection.commit()
        print("Dados inseridos com sucesso!")

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao inserir dados: {error.message}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {str(e)}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [706]:
# Inserindo os dados
if connection:
    insert_municipio_data(connection, gdf_mun)

Dados inseridos com sucesso!
Cursor fechado.


In [707]:
def read_municipio_data(connection):
    cursor = connection.cursor()
    
    try:
        # Consultar dados da tabela
        cursor.execute("""
            SELECT 
                CD_MUN, 
                NM_MUN, 
                SG_Estado, 
                AREA_KM2, 
                SDO_UTIL.TO_WKTGEOMETRY(GEOMETRY) as geometry_wkt, 
                SDO_UTIL.TO_WKTGEOMETRY(CORD_CENTRAL) as cord_central_wkt 
            FROM TBL_Municipio
        """)
        
        rows = cursor.fetchall()

        # Extrair as colunas dos resultados
        col_names = [desc[0] for desc in cursor.description]

        # Criar um DataFrame com os resultados
        df = pd.DataFrame(rows, columns=col_names)

        # Converter colunas LOB para string e remover as colunas extras
        df['geometry'] = df['GEOMETRY_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else loads(x))
        df['CORD_CENTRAL'] = df['CORD_CENTRAL_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else (loads(x) if x else None))

        # Remover colunas WKT que não são mais necessárias
        df.drop(columns=['GEOMETRY_WKT', 'CORD_CENTRAL_WKT'], inplace=True)

        # Criar um GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='geometry')

        return gdf

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao ler dados: {error.message}")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [708]:
# Ler os dados da tabela
Select_TBL_Municipio = read_municipio_data(connection)
Select_TBL_Municipio

Cursor fechado.


,CD_MUN,NM_MUN,SG_ESTADO,AREA_KM2,geometry,CORD_CENTRAL
0,3550308,São Paulo,SP,1521.11,"POLYGON ((-46.55066 -23.35714, -46.55011 -23.3...",POINT (-46.647972 -23.649892)


##### TBL_Zona 

In [709]:
# Consulta SQL
query_teste = "SELECT * FROM TBL_ZONA"

# Executa a consulta e obtém os resultados em um DataFrame
df_teste = consulta_para_dataframe(cursor, query_teste)
df_teste

,ID,NOME_ZONA,GEOMETRY,CORD_CENTRAL,MUNICIPIO_ID


In [710]:
# Carregar o arquivo GeoJSON usando GeoPandas
gdf_zon = gpd.read_file(DIR_ZONAS_SP)

# Verifique o conteúdo do GeoDataFrame
print(gdf_zon.head())
print(gdf_zon.dtypes)

              Zona                   CORD_CENTRAL   CD_MUN  \
0  Centro Ampliado  POINT (-46.679579 -23.582388)  3550308   
1     Zona Leste 1  POINT (-46.542663 -23.561831)  3550308   
2     Zona Leste 2  POINT (-46.443353 -23.549433)  3550308   
3       Zona Norte   POINT (-46.673093 -23.45335)  3550308   
4         Zona Sul  POINT (-46.708599 -23.813758)  3550308   

                                            geometry  
0  POLYGON ((-46.67421 -23.64222, -46.67437 -23.6...  
1  POLYGON ((-46.57536 -23.6008, -46.57552 -23.60...  
2  POLYGON ((-46.45643 -23.64132, -46.4565 -23.64...  
3  POLYGON ((-46.67805 -23.51367, -46.67863 -23.5...  
4  POLYGON ((-46.60925 -23.90462, -46.60876 -23.9...  
Zona              object
CORD_CENTRAL      object
CD_MUN             int32
geometry        geometry
dtype: object


In [711]:
gdf_zon

,Zona,CORD_CENTRAL,CD_MUN,geometry
0,Centro Ampliado,POINT (-46.679579 -23.582388),3550308,"POLYGON ((-46.67421 -23.64222, -46.67437 -23.6..."
1,Zona Leste 1,POINT (-46.542663 -23.561831),3550308,"POLYGON ((-46.57536 -23.6008, -46.57552 -23.60..."
2,Zona Leste 2,POINT (-46.443353 -23.549433),3550308,"POLYGON ((-46.45643 -23.64132, -46.4565 -23.64..."
3,Zona Norte,POINT (-46.673093 -23.45335),3550308,"POLYGON ((-46.67805 -23.51367, -46.67863 -23.5..."
4,Zona Sul,POINT (-46.708599 -23.813758),3550308,"POLYGON ((-46.60925 -23.90462, -46.60876 -23.9..."


In [712]:
def insert_zona_data(connection, gdf_zon):
    cursor = connection.cursor()

    try:
        for _, row in gdf_zon.iterrows():
            nome = row['Zona']
            municipio_id = row['CD_MUN']  # Presumindo que CD_MUN refere-se ao MUNICIPIO_ID

            # Certifique-se de que as colunas 'geometry' e 'CORD_CENTRAL' são objetos Shapely
            if isinstance(row['geometry'], str):
                geom = loads(row['geometry'])
            else:
                geom = row['geometry']

            if isinstance(row['CORD_CENTRAL'], str):
                central_geom = loads(row['CORD_CENTRAL'])
            else:
                central_geom = row['CORD_CENTRAL']

            # Converta para WKT para ser utilizado no PL/SQL
            geometry_wkt = geom.wkt
            cor_central_wkt = central_geom.wkt if central_geom else None

            # Bloco PL/SQL para inserir a geometria no Oracle
            plsql = """
            DECLARE
                geom SDO_GEOMETRY;
                central_geom SDO_GEOMETRY;
            BEGIN
                geom := SDO_GEOMETRY(:geometry_wkt, 4326);
                IF :cor_central_wkt IS NOT NULL THEN
                    central_geom := SDO_GEOMETRY(:cor_central_wkt, 4326);
                ELSE
                    central_geom := NULL;
                END IF;
                INSERT INTO TBL_ZONA (NOME_ZONA, GEOMETRY, CORD_CENTRAL, MUNICIPIO_ID)
                VALUES (:nome, geom, central_geom, :municipio_id);
            END;
            """
            cursor.execute(plsql, nome=nome, municipio_id=municipio_id, 
                           geometry_wkt=geometry_wkt, cor_central_wkt=cor_central_wkt)

        connection.commit()
        print("Dados inseridos com sucesso na TBL_ZONA!")

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao inserir dados: {error.message}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {str(e)}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [713]:
if connection:
    insert_zona_data(connection, gdf_zon)

Dados inseridos com sucesso na TBL_ZONA!
Cursor fechado.


In [714]:
def read_zona_data(connection):
    cursor = connection.cursor()
    
    try:
        # Consultar dados da tabela TBL_ZONA
        cursor.execute("""
            SELECT 
                ID, 
                NOME_ZONA, 
                MUNICIPIO_ID,
                SDO_UTIL.TO_WKTGEOMETRY(GEOMETRY) as geometry_wkt, 
                SDO_UTIL.TO_WKTGEOMETRY(CORD_CENTRAL) as cord_central_wkt 
            FROM TBL_ZONA
        """)
        
        rows = cursor.fetchall()

        # Extrair as colunas dos resultados
        col_names = [desc[0] for desc in cursor.description]

        # Criar um DataFrame com os resultados
        df = pd.DataFrame(rows, columns=col_names)

        # Converter colunas LOB para string e remover as colunas extras
        df['geometry'] = df['GEOMETRY_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else loads(x))
        df['CORD_CENTRAL'] = df['CORD_CENTRAL_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else (loads(x) if x else None))

        # Remover colunas WKT que não são mais necessárias
        df.drop(columns=['GEOMETRY_WKT', 'CORD_CENTRAL_WKT'], inplace=True)

        # Criar um GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='geometry')

        return gdf

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao ler dados: {error.message}")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [715]:
# Ler os dados da tabela
Select_TBL_Zona = read_zona_data(connection)
Select_TBL_Zona

Cursor fechado.


,ID,NOME_ZONA,MUNICIPIO_ID,geometry,CORD_CENTRAL
0,1,Centro Ampliado,3550308,"POLYGON ((-46.67421 -23.64222, -46.67437 -23.6...",POINT (-46.679579 -23.582388)
1,2,Zona Leste 1,3550308,"POLYGON ((-46.57536 -23.6008, -46.57552 -23.60...",POINT (-46.542663 -23.561831)
2,3,Zona Leste 2,3550308,"POLYGON ((-46.45643 -23.64132, -46.4565 -23.64...",POINT (-46.443353 -23.549433)
3,4,Zona Norte,3550308,"POLYGON ((-46.67805 -23.51367, -46.67863 -23.5...",POINT (-46.673093 -23.45335)
4,5,Zona Sul,3550308,"POLYGON ((-46.60925 -23.90462, -46.60876 -23.9...",POINT (-46.708599 -23.813758)


##### TBL_Distrito

In [716]:
# Consulta SQL
query_teste = "SELECT * FROM TBL_DISTRITO"

# Executa a consulta e obtém os resultados em um DataFrame
df_teste = consulta_para_dataframe(cursor, query_teste)
df_teste

,CD_DIST,NM_DIST,CORD_CENTRAL,ZONA_ID


In [717]:
# Carregar o arquivo GeoJSON usando GeoPandas
gdf_dist = gpd.read_file(DIR_DISTRITOS_SP)

In [718]:
# selecionando apenas as colunas de interesse
Select_TBL_Zona_selecao = Select_TBL_Zona[['ID', 'NOME_ZONA']].rename(columns={'ID': 'ID_ZONA'})

In [719]:
# Fazer o merge das duas tabelas usando a coluna 'Zona'
gdf_dist_merged = gdf_dist.merge(Select_TBL_Zona_selecao, on='NOME_ZONA', how='left')

In [720]:
# Dropando coluna de geometrias para poupar espaçamento
gdf_dist_merged = gdf_dist_merged.drop(columns={"geometry"})

In [721]:
# Verifique o conteúdo do GeoDataFrame
print(gdf_dist_merged.head())
print(gdf_dist_merged.dtypes)

    CD_MUN     NM_MUN    CD_DIST            NM_DIST  \
0  3550308  São Paulo  355030801          Água Rasa   
1  3550308  São Paulo  355030802  Alto de Pinheiros   
2  3550308  São Paulo  355030803         Anhanguera   
3  3550308  São Paulo  355030804         Aricanduva   
4  3550308  São Paulo  355030805        Artur Alvim   

                    CORD_CENTRAL        NOME_ZONA  ID_ZONA  
0   POINT (-46.571849 -23.56688)     Zona Leste 1        2  
1  POINT (-46.711886 -23.547577)  Centro Ampliado        1  
2  POINT (-46.792944 -23.430469)       Zona Norte        4  
3  POINT (-46.515679 -23.573925)     Zona Leste 1        2  
4  POINT (-46.485831 -23.540212)     Zona Leste 1        2  
CD_MUN          object
NM_MUN          object
CD_DIST         object
NM_DIST         object
CORD_CENTRAL    object
NOME_ZONA       object
ID_ZONA          int64
dtype: object


In [724]:
def insert_distrito_data(connection, gdf):
    cursor = connection.cursor()

    try:
        for _, row in gdf.iterrows():
            cd_dist = row['CD_DIST']
            nm_dist = row['NM_DIST']
            zona_id = row['ID_ZONA']

            # Certifique-se de que a coluna 'CORD_CENTRAL' é um objeto Shapely
            if isinstance(row['CORD_CENTRAL'], str):
                central_geom = loads(row['CORD_CENTRAL'])
            else:
                central_geom = row['CORD_CENTRAL']

            # Converta para WKT para ser utilizado no PL/SQL
            cor_central_wkt = central_geom.wkt if central_geom else None

            # Bloco PL/SQL para inserir a coordenada central no Oracle
            plsql = """
            DECLARE
                central_geom SDO_GEOMETRY;
            BEGIN
                IF :cor_central_wkt IS NOT NULL THEN
                    central_geom := SDO_GEOMETRY(:cor_central_wkt, 4326);
                ELSE
                    central_geom := NULL;
                END IF;
                INSERT INTO TBL_DISTRITO (CD_DIST, NM_DIST, CORD_CENTRAL, ZONA_ID)
                VALUES (:cd_dist, :nm_dist, central_geom, :zona_id);
            END;
            """
            cursor.execute(plsql, cd_dist=cd_dist, nm_dist=nm_dist, zona_id=zona_id, cor_central_wkt=cor_central_wkt)

        connection.commit()
        print("Dados inseridos com sucesso na tabela TBL_DISTRITO!")

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao inserir dados: {error.message}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {str(e)}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [725]:
if connection:
    insert_distrito_data(connection, gdf_dist_merged)

Erro ao inserir dados: ORA-01438: value larger than specified precision allowed for this column
ORA-06512: at line 10
Help: https://docs.oracle.com/error-help/db/ora-01438/
Alterações desfeitas (rollback) devido ao erro.
Cursor fechado.


#### INSERT DADOS - TBL_PREVISAO